<a href="https://colab.research.google.com/github/21cs076/Real-Time-Landslide-Detection-and-Buried-Vehicle-Identification-using-YOLOv8/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi  # For GPU

In [ ]:
# Install Ultralytics package
!pip install ultralytics
!pip install kagglehub

from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('yolov8n.pt')

# Perform inference on an image from Google Drive
results = model('https://ultralytics.com/images/bus.jpg')

# Iterate through the results and display each
for result in results:
    result.show()

In [ ]:
import shutil
import os

# Define the path to the dataset
dataset_path = '/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge'

# Check if the directory exists
if os.path.exists(dataset_path):
    # Remove the directory and its contents
    shutil.rmtree(dataset_path)
    print(f"The dataset at {dataset_path} has been deleted.")
else:
    print(f"The dataset at {dataset_path} does not exist.")

import kagglehub
import os

# Replace the dataset with Landslide4Sense Challenge
path = kagglehub.dataset_download("pypiahmad/landslide4sense-challenge")

print("Path to dataset files:", path)

In [ ]:
import h5py
import numpy as np
from PIL import Image
import os
import shutil

# Function to convert .h5 images to .jpg
def convert_h5_images_to_jpg(h5_file_path, save_dir):
    with h5py.File(h5_file_path, 'r') as h5f:
        img_data = h5f['img'][:]  # Assuming 'img' is the key for image data

        # Print the shape of the data for debugging
        print(f"Original shape: {img_data.shape}, dtype: {img_data.dtype}")

        # Select the first three channels for RGB
        img_data = img_data[:, :, :3]

        # Normalize and convert the data to uint8
        img_data = (img_data * 255).astype(np.uint8)

        # Ensure data is in RGB format
        img = Image.fromarray(img_data, mode='RGB')

        # Create the images directory if it doesn't exist
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        img_name = os.path.splitext(os.path.basename(h5_file_path))[0] + '.jpg'
        img.save(os.path.join(save_dir, img_name))

# Function to convert .h5 masks to .txt in YOLO format
def convert_h5_masks_to_txt(h5_file_path, save_dir, img_nm):
    with h5py.File(h5_file_path, 'r') as h5f:
        mask_data = h5f['mask'][:]  # Assuming 'mask' is the key for mask data
        txt_name = img_nm + '.txt'
        #label_dir = os.path.join(save_dir, 'labels')

        # Create the labels directory if it doesn't exist
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        with open(os.path.join(save_dir, txt_name), 'w') as f:
            for mask in mask_data:
                # Example: Convert mask to YOLO format and write to txt
                # Assuming mask is [class, x_center, y_center, width, height]
                yolo_format = ' '.join(map(str, mask))
                f.write(yolo_format + '\n')

# Define input paths (updated dataset locations)
train_img_path = '/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/TrainData/img'
train_mask_path = '/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/TrainData/mask'
test_img_path = '/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/TestData/img'
valid_img_path = '/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/ValidData/img'

# Define output paths for each dataset (images and labels separately)
train_output_path = '/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/TrainData'
test_output_path = '/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/TestData'
valid_output_path = '/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/ValidData'

# Process TrainData img and mask
img_files = [f for f in os.listdir(train_img_path) if f.startswith('image') and f.endswith('.h5')]
mask_files = [f for f in os.listdir(train_mask_path) if f.startswith('mask') and f.endswith('.h5')]

# Ensure both lists are sorted for consistent pairing
img_files.sort()
mask_files.sort()

for img_file, mask_file in zip(img_files, mask_files):
    # Convert images from .h5 to .jpg
    convert_h5_images_to_jpg(os.path.join(train_img_path, img_file), os.path.join(train_output_path, 'images'))

    # Convert masks from .h5 to .txt
    img_nm = os.path.splitext(os.path.basename(os.path.join(train_img_path, img_file)))[0]
    convert_h5_masks_to_txt(os.path.join(train_mask_path, mask_file), os.path.join(train_output_path, 'labels'), img_nm)

# Process TestData img
for file_name in os.listdir(test_img_path):
    if file_name.startswith('image') and file_name.endswith('.h5'):
        convert_h5_images_to_jpg(os.path.join(test_img_path, file_name), os.path.join(test_output_path, 'images'))

# Process ValidData img
for file_name in os.listdir(valid_img_path):
    if file_name.startswith('image') and file_name.endswith('.h5'):
        convert_h5_images_to_jpg(os.path.join(valid_img_path, file_name), os.path.join(valid_output_path, 'images'))

# Optional: Delete img and mask folders after conversion
shutil.rmtree(train_img_path)
shutil.rmtree(train_mask_path)
shutil.rmtree(test_img_path)
shutil.rmtree(valid_img_path)


In [ ]:
import shutil
from google.colab import files

# Define the source and target paths
dataset_path = "/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1"
zip_file_path = "/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/landslide4sense_dataset.zip"  # Adjust as needed for your environment

# Compress the dataset folder into a zip file
shutil.make_archive(base_name=zip_file_path.replace('.zip', ''), format='zip', root_dir=dataset_path)
print(f"Dataset compressed and saved to: {zip_file_path}")

# Download the ZIP file
files.download(zip_file_path)


In [ ]:
print(os.listdir(path))
print(os.listdir('/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/TrainData/images'))
print(os.listdir('/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/TrainData/labels'))
print(os.listdir('/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/ValidData/images'))
print(os.listdir('/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/TestData/images'))

In [ ]:
import os

image_dir = '/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/TrainData/images'
label_dir = '/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/TrainData/labels'
images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

for image in images:
    label_file = os.path.splitext(image)[0] + '.txt'
    if not os.path.exists(os.path.join(label_dir, label_file)):
        print(f"Missing label for image: {image}")


yaml_content = """
train: /root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/TrainData
val: /root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/ValidData
test: /root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/TestData

nc: 2  # Number of classes (e.g., landslide, non-landslide)
names: ['landslide', 'non-landslide']
"""

# Save the content to a .yaml file
with open("/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/dataset.yaml", "w") as file:
    file.write(yaml_content)

# Verify the file was created successfully
with open("/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/dataset.yaml", "r") as file:
    print(file.read())


In [ ]:
from ultralytics import YOLO

# Initialize model
model = YOLO('yolov8n.pt')  # Use pre-trained weights

# Define the training configuration
epochs = 100  # You can adjust this as needed
data_path = '/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/dataset.yaml'
imgsz = 640

# Train the model
#model.train(data=data_path, epochs=epochs, imgsz=imgsz)
model.train(data=data_path, epochs=epochs, imgsz=imgsz, val=False)

# Save the trained model in PyTorch format
model.save('/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/best_model.pt')

# Export the model to ONNX format
model.export(format='onnx', path='/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/best_model.onnx')

In [ ]:
import onnxruntime as ort
import numpy as np

# Load the exported ONNX model using onnxruntime for inference
onnx_model_path = '/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/best_model.onnx'
onnx_session = ort.InferenceSession(onnx_model_path)

# Inference example using ONNX model (dummy input)
# Assuming input is a 640x640 image
dummy_input = np.random.rand(1, 3, 640, 640).astype(np.float32)

# Run inference
outputs = onnx_session.run(None, {'images': dummy_input})

# Print the outputs (you can process it based on your needs)
print(outputs)


In [ ]:

# Perform inference
results = model('/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/TestData/image_1.jpg')

# Display results
results.show()


In [ ]:
# Clone YOLOv8 repository
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics
